In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9975586455790256, 0.997281482927443)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9995600008411749, 0.9995612392705429)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=10)

In [9]:
y

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
0,ItalianFiscalCode,ItalianVAT,NaN,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Error,NaN
1,ItalianFiscalCode,ItalianVAT,NaN,NaN,Plate,Address,NaN,NaN,NaN,NaN,...,CountryCode,Name,Surname,NaN,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,NaN
2,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,NaN,...,CountryCode,Name,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
3,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Plate,Address,NaN,ProvinceCode,Region,NaN,...,CountryCode,NaN,NaN,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
4,ItalianFiscalCode,NaN,CadastreCode,NaN,NaN,Address,ItalianZIPCode,NaN,Error,NaN,...,NaN,Name,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
5,ItalianFiscalCode,NaN,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,NaN,String,EMail,PhoneNumber,Date,Error,Boolean,NumericId
6,ItalianFiscalCode,NaN,CadastreCode,Document,Plate,Address,NaN,NaN,Region,NaN,...,CountryCode,NaN,NaN,String,NaN,NaN,Date,BiologicalSex,NaN,NaN


In [10]:
model.predict_dataframe(X)

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
0,ItalianFiscalCode,ItalianVAT,NaN,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Error,NaN
1,ItalianFiscalCode,ItalianVAT,NaN,NaN,Plate,Address,NaN,NaN,NaN,NaN,...,CountryCode,Name,Surname,NaN,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,NaN
2,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,NaN,...,CountryCode,Name,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
3,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Plate,Address,NaN,ProvinceCode,Region,NaN,...,CountryCode,NaN,NaN,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
4,ItalianFiscalCode,NaN,CadastreCode,NaN,NaN,Address,ItalianZIPCode,NaN,Error,NaN,...,NaN,Name,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
5,ItalianFiscalCode,NaN,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,NaN,String,EMail,PhoneNumber,Date,Error,Boolean,NumericId
6,ItalianFiscalCode,NaN,CadastreCode,Document,Plate,Address,NaN,NaN,Region,NaN,...,CountryCode,NaN,NaN,String,NaN,NaN,Date,BiologicalSex,NaN,NaN


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('ItalianZIPCode', 'Error'): 8,
         ('Error', 'Document'): 141,
         ('CountryCode', 'NaN'): 72,
         ('Error', 'ItalianVAT'): 14,
         ('CountryCode', 'ProvinceCode'): 6,
         ('NumericId', 'Integer'): 28,
         ('NaN', 'CountryCode'): 96,
         ('Error', 'NumericId'): 59,
         ('Error', 'Boolean'): 6,
         ('ItalianZIPCode', 'NumericId'): 339,
         ('String', 'Error'): 170,
         ('Error', 'EMail'): 6,
         ('Error', 'CountryCode'): 22,
         ('ProvinceCode', 'CountryCode'): 5,
         ('Name', 'Error'): 13,
         ('Document', 'Error'): 22,
         ('Float', 'Error'): 6,
         ('NumericId', 'ItalianZIPCode'): 308,
         ('Error', 'ItalianFiscalCode'): 19,
         ('Error', 'ProvinceCode'): 23,
         ('Error', 'String'): 19,
         ('NumericId', 'Error'): 26,
         ('Error', 'Municipality'): 3,
         ('Error', 'Country'): 3,
         ('Integer', 'NumericId'): 8,
         ('Integer', 'Error'): 12,
        